# Jupyter Notebook程序设计：朗肯循环计算

## 问题描述

![ReheatRankineCircle](img/ReheatRankineCircle.jpg)

本系统为再热朗肯循环，具有两个给水加热器：封闭给水加热器和开放给水加热器。

* 蒸汽在8.0 MPa，480℃下进入第一台汽轮机，膨胀至0.7 MPa。
* 蒸汽在进入第二台汽轮机之前被重新加热至440 ℃，膨胀至凝汽器压力0.008 MPa。
* 蒸汽以2 MPa的压力从第一台汽轮机抽取，并送入封闭的给水加热器。
* 给水在205℃和8.0MPa下离开封闭给水加热器，冷凝物以2 MPa作为饱和水离开。
* 凝结水被捕获在开放的给水加热器中。
* 从第二台汽轮机以0.3 MPa抽取的蒸汽也被送入开放给水加热器，其工作压力为0.3 MPa。
* 离开开放给水加热器的为0.3 MPa下的饱和水。
* 该循环的净功率输出为100 MW。
* 没有从任何部件到其周围的杂散热传递。
* 工作流体在通过汽轮机，泵，锅炉，再热器和凝汽器时可逆。

已知条件与求解目标：
* 已知：再热再生蒸汽动力循环使用蒸汽作为工作流体。规定了工作压力和温度，给出了净功率输出。
* 求解：确定进入第一台汽轮机的热效率和质量流量，单位为kg / h。

## 问题分析与解答

根据已知条件，第一步应该是求解出整个循环中13个状态点的水蒸汽和水的物行参数。

我们知道，水蒸气的状态参数包括了：压力、温度、焓、熵、干度、比容。每一个状态点都包括了这六个特性，而且这六个特性之间彼此相关联，已知两个就可以调用封装好的函数求解其余特性。因此，可以将单位千克的给水（feedwater）设置成一个类（class），类中包括了六个属性（attribute）。类中的函数作为初始化的功能，可以根据已知条件进行调用。

这个类作为一个模块，单独编写在一个.py的文件下，这样在解决其他与水蒸气物性相关的问题时，直接导入（import）即可，不用重新编写。这也是初始化函数写得比较全面的原因。虽然本题中可能用不到比容的特性，也不会调用所有的函数，但是考虑到更广泛的适用性，可以在这个模块里多写一些。

参考老师给的SEUIF97里python的简化版朗肯循环程序，发现这一个类确实可以直接导入，这就是模块化的特点之一：各个模块互不干扰，装拆适用性很强。

In [ ]:
# %load Feedwater.py
from seuif97 import *


class Feedwater(object):
    '''
    Initializes the feedwater quality with different conditions
    '''
    
    def __init__(self):
        self.p = None
        self.t = None
        self.h = None
        self.s = None
        self.x = None
        self.v = None

    def pt(self):
        self.h = pt2h(self.p, self.t)
        self.s = pt2s(self.p, self.t)
        self.v = pt2v(self.p, self.t)
        self.x = None

    def ph(self):
        self.t = ph2t(self.p, self.h)
        self.s = ph2s(self.p, self.h)
        self.v = ph2v(self.p, self.h)
        self.x = ph2x(self.p, self.h)

    def ps(self):
        self.t = ps2t(self.p, self.s)
        self.h = ps2h(self.p, self.s)
        self.v = ps2v(self.p, self.s)
        self.x = ps2x(self.p, self.s)

    def hs(self):
        self.t = hs2t(self.h, self.s)
        self.p = hs2p(self.h, self.s)
        self.v = hs2v(self.h, self.s)
        self.x = hs2x(self.h, self.s)

    def px(self):
        self.t = px2t(self.p, self.x)
        self.h = px2h(self.p, self.x)
        self.s = px2s(self.p, self.x)
        self.v = px2v(self.p, self.x)

    def tx(self):
        self.p = tx2p(self.t, self.x)
        self.h = tx2h(self.t, self.x)
        self.s = tx2s(self.t, self.x)
        self.v = tx2v(self.t, self.x)


***
有了状态点的类之后，并不能直接求出各个状态所有参数，因为很多状态点只有一个参数已知。

但是，由于这是一个理想的循环，两个相邻的状态点之间是被过程连接，参数之间也是有联系的。比如：
* 给水在锅炉内是等压升温吸热。　
* 过热蒸汽在汽轮机内看作等熵做功。　
* 凝汽器内压力不变，蒸汽凝结成饱和水。　
* 凝结水在泵内等熵升压。　
* 开放给水加热器进口焓之和等于出口焓。　
* 封闭给水加热器蒸汽放热量与给水吸热量相等。　

因此，我们可以把各个过程也看作一个模块，将它们设成一个类，连接进出口参数。

题目中，汽轮机虽然只分成两个部分，但由于有抽汽回热，因此可以将汽轮机看作四个部分。一个凝汽器，两个泵。锅炉可看作两个，一个是加热主蒸汽，另一个加热再热蒸汽。其中为了后续计算方便，在锅炉类内增加一个元素，表示单位千克给水吸收的热量（用焓值变化表示）；在泵类内增加一个元素，表示单位千克给水升压消耗的热量；在汽轮机类内增加一个元素，表示单位千克蒸汽做功量。

In [ ]:
# %load Boiler.py
import Feedwater


class Boiler:
    '''
    The boiler class
    '''    
   
    def __init__(self,inlet,outlet):
        '''
        Initializes the boiler with the previous conditions
        '''
        self.inlet = inlet
        self.outlet = outlet

    def simulate(self):
        '''
        Simulates the Boiler and tries to get the exit temperature 
        down to the desiredOutletTemp. This is done by continuously
        adding h while keeping the P constant.
        '''
        self.outlet.p = self.inlet.p
        self.outlet.pt()
        self.Addh = self.outlet.h - self.inlet.h

In [ ]:
# %load Turbine.py
import Feedwater
      
class Turbine:
    '''
    Turbine class
    '''
    
    def __init__(self,inlet,outlet):
        '''
        Initializes the turbine with the conditions
        '''
        self.inlet = inlet
        self.outlet = outlet   
    
    def simulate(self):
        """
        Simulates the turbine and tries to have the exit quality
        as desired. It isentropically work from the turbine until 
        the desired outlet quality is reached.
        """
        self.outlet.s = self.inlet.s
        self.outlet.ps()
        self.Extracth = self.inlet.h - self.outlet.h
             

In [ ]:
# %load Condenser.py
import Feedwater


class Condenser:
    '''
    The condenser class
    '''   
   
    def __init__(self,inlet,outlet):
        '''
        Initializes the condenser with the conditions
        '''
        self.inlet = inlet
        self.outlet = outlet
        
    def simulate(self,overcool):
        '''
        Simulates the Condenser and tries to get the exit temperature
        down to the desired outlet temprature. This is done by 
        continuously extracting h while keeping the P constant.
        '''
        self.Extracth = self.inlet.h - self.outlet.h
        self.outlet.h = self.outlet.h - overcool
        self.outlet.ph()
        

In [ ]:
# %load Pump.py
import Feedwater


class Pump:
    '''
    Pump class
    '''   

    def __init__(self,inlet,outlet):
        '''
        Initializes the pump with the conditions
        '''
        self.inlet = inlet
        self.outlet = outlet
        
    def simulate(self):
        '''
        Simulates the pump and tries to have the exit quality
        as desired. It isentropically work from the pump until 
        the desired outlet quality is reached.
        '''
        self.outlet.s = self.inlet.s
        self.outlet.ps()
        self.Needh = self.outlet.h - self.inlet.h
        

要注意的是，以上所有类都用到了给水类，作为自己的特性，所以需要在最前面导入给水类。
***

下面是解决问题的程序的主体部分。

主程序的计算函数其实也分了三个部分：初始化，模拟循环和计算结果。

**初始化部分**较为繁琐，首先生成状态点的一个list，然后要根据题目给出条件将每个状态点的已知条件写出，如果遗漏则可能无法求解。每个过程中可能有的等熵之类的特点不在这里考虑，因为在过程类中已经利用过这些条件。

**模拟部分**有了前面的基础，模拟就是利用过程求出所有状态点的所有特性，注意求解顺序和语法即可。

**计算结果部分**有了各个状态点的参数，就相当于以前求解时，焓值的表已经查找完了，剩下的就是针对题目要求的进行计算，然后显示出来。两个给水加热器利用能量守恒可以求出y'和y''，然后效率也就可以求得了。根据发电功率，也可以计算得到需要的蒸汽量。13个状态点的参数用表格的方式显示比较清晰，不过首先要安装tabulate的软件包。

In [1]:
# %load ReheatRankineCircle.py
from tabulate import tabulate
import Feedwater
import Boiler
import Turbine
import Condenser
import Pump


def ReheatRankineCirlce():
    '''
    Solution to the example 8.6
    '''
    
    # Initialize
    W = 100
    condenserOverCool = 0
    condenserPressure = 0.008
    enterPressure = 8.0
    enterTemp = 480
    reheatTemp = 440
    extractFirstTurbinePressure = 2.0
    exitFirstTurbinePressure = 0.7
    exitClosedHeaterTemp = 205
    openHeaterPressure = 0.3
    
    table = []
   
    states = []
    for i in range(13):
        states.append(Feedwater.Feedwater())
        
    states[0].p = enterPressure
    states[0].t = enterTemp
    
    states[1].p = extractFirstTurbinePressure

    states[2].p = exitFirstTurbinePressure
    
    states[3].t = reheatTemp
    
    states[4].p = openHeaterPressure
    
    states[5].p = condenserPressure
    
    states[6].p = condenserPressure
    states[6].x = 0
    
    states[7].p = openHeaterPressure
    
    states[8].p = openHeaterPressure
    states[8].x = 0
    
    states[9].p = enterPressure
    
    states[10].p = enterPressure
    states[10].t = exitClosedHeaterTemp
    
    states[11].x = 0
    states[11].p = extractFirstTurbinePressure
    
    states[12].p = openHeaterPressure 
    
    # Simulate
    states[0].pt()
    t1 = Turbine.Turbine(states[0], states[1])  


    t1.simulate()
    states[1] = t1.outlet
    
    t2 = Turbine.Turbine(states[1],states[2])
    t2.simulate()
    states[2] = t2.outlet
    
    b1 = Boiler.Boiler(states[2], states[3])
    b1.simulate()
    states[3] = b1.outlet
    
    t3 = Turbine.Turbine(states[3], states[4]) 
    t3.simulate()
    states[4] = t3.outlet
    
    t4 = Turbine.Turbine(states[4], states[5]) 
    t4.simulate()
    states[5] = t4.outlet
    
    states[6].px()
    c = Condenser.Condenser(states[5],states[6])
    c.simulate(condenserOverCool)
    
    p1 = Pump.Pump(states[6],states[7])
    p1.simulate()
    states[7] = p1.outlet
    
    states[8].px()
    
    p2 = Pump.Pump(states[8],states[9])
    p2.simulate()
    states[9] = p2.outlet
    
    states[10].pt()
    
    states[11].px()
    
    states[12].h = states[11].h    #Saturated water throttle, enthalpy unchanged
    states[12].ph()
    
    b2 = Boiler.Boiler(states[10],states[0])
    b2.simulate()
    
    # Results
    for i in range(13):
        table.append([i+1, states[i].h, states[i].s, states[i].p, states[i].t] )
    print(tabulate(table, headers=["State", "Enthalpy", "Etropy", "Pressure", "Temperature"]))
    
    y1 = (states[10].h-states[9].h) / (states[1].h-states[11].h)
    y2 = (states[8].h - y1*states[12].h - (1-y1)*states[7].h) / (states[4].h - states[7].h)
    
    output = t1.Extracth + (1 - y1) * t2.Extracth + (1 - y1) * t3.Extracth + (1 - y1 - y2) * t4.Extracth\
             - (1 - y1 - y2) * p1.Needh - p2.Needh
    input = (1 - y1) * b1.Addh + b2.Addh
    
    efficiency = output / input
    
    m = W * 1000000 * 3600 / (1000 * output)
    
    print('\n', "y1 = ",y1, ", y2 = ", y2)
    print('\n',"Efficiency = ", efficiency)
    print('\n',"MassFlow = ", m, "kg/h")
   
    
if __name__ == '__main__':
    ReheatRankineCirlce()

  State    Enthalpy    Etropy    Pressure    Temperature
-------  ----------  --------  ----------  -------------
      1    3349.53   6.66106        8           480
      2    2964.05   6.66106        2           274.521
      3    2742.63   6.66106        0.7         164.953
      4    3353.81   7.75881        0.7         440
      5    3101.62   7.75881        0.3         315.687
      6    2428.79   7.75881        0.008        41.5101
      7     173.852  0.592532       0.008        41.5101
      8     174.146  0.592532       0.3          41.5188
      9     561.455  1.67176        0.3         133.525
     10     569.704  1.67176        8           134.252
     11     877.41   2.36772        8           205
     12     908.622  2.44702        2           212.385
     13     908.622  2.52543        0.3         133.525

 y1 =  0.14970428650697784 , y2 =  0.09474213093251493

 Efficiency =  0.43053435881573765

 MassFlow =  279487.57433731604 kg/h


## 工作小结

在本次的练习过程中，我比较清晰的认识了 *面向对象编程* 和 *程序模块化* 的意义。开始看到老师给的好多个程序文件，都不知道应该运行哪一个，但是在学习class之后，知道了如何导入模块，结构就一下子清晰了。

虽然有老师的程序作为参考，也用着计算水蒸气特性的封装好的程序，可以说站在了巨人的肩膀上，但本次练习也不是完全顺利的。在模拟的部分中，我遇到了一个报错：我试图把调用过函数的一个对象（初始化各个参数之后），赋值给它自己。

` states[0] = states[0].pt() `

` AttributeError: 'NoneType' object has no attribute 's' ` 

print了几次，等号右边是Feedwater数据类型，赋值之前的states[0]也是Feedwater数据类型，为什么赋值之后就变成了None的类型呢？

后来我发现是因为pt()这个函数没有设置返回值。它虽然可以修改states[0]内的属性值，可是返回值是None，解决问题的办法要么是加一个返回值，要么是不用赋值，直接调用函数就已经修改了这个状态点的所有参数。

对于知识的运用比了解这个知识重要得多。将工程热力学的知识用程序展现出来也有助于我更好的掌握它，完成这一项作业还是比较有成就感的。不知道给水加热器是否可以也设置成一个类，在我的程序中，这两个加热器由于特点不同，就没有抽象成一个类来进行描述，而是在主程序中用它们各自的特点直接求解抽汽量。

ps：Jupyter里面有些关键词会标红，比如Turbine.py里的class，在别的地方显示都没有问题，只有这个cell的这一个关键词变红了。老师的课件里也出现过几次，原因是Jupyter的语法高亮，多了一个空行，删去即可，已更正。

## 参考资料

* [SEUIF97/PyRankine](https://github.com/PySEE/SEUIF97/tree/master/PyRankine)
* [Markdown——入门指南](http://www.jianshu.com/p/1e402922ee32/)
* [ReasonofAttributeError](http://stackoverflow.com/questions/8949252/python-attribute-error-nonetype-object-has-no-attribute-something)
* [FUNDAMENTALS OF ENGINEERING THERMODYNAMICS](http://zh.bookzz.org/book/1294112/a61ecf)